In [2]:
!pip install git+https://github.com/facebookresearch/ImageBind.git
!pip install faiss-cpu torch torchaudio torchvision

  Cloning https://github.com/facebookresearch/ImageBind.git to /tmp/pip-req-build-gmsubq5n
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ImageBind.git /tmp/pip-req-build-gmsubq5n
  Resolved https://github.com/facebookresearch/ImageBind.git to commit cfee5753e8e0b7a9df8aeb19c493e8facd81a5f2
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 6cdc929315aab1b5674b6dcf73b16ec99147735f) to /tmp/pip-install-xwofpq1w/pytorchvideo_e2de360f596d4d6c91e13b6fced87cec
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-xwofpq1w/pytorchvideo_e2de360f596d4d6c91e13b6fced87cec
  Running command git rev-parse -q --verify 'sha^6cdc929315aab1b5674b6dcf73b16ec99147735f'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 6cdc929315aab1b5674b6dcf73b16ec99147735f
  Running command git checkout -q 

In [3]:
!pip install sympy==1.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.8.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.


In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')

SAVE_DIR = "/content/drive/MyDrive/Colab Notebooks/Context_Aware_Audio"
os.makedirs(SAVE_DIR, exist_ok=True)

print("Project directory:", SAVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project directory: /content/drive/MyDrive/Colab Notebooks/Context_Aware_Audio


In [5]:
!mkdir /content/test_scene

In [6]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-kwwrug9s
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-kwwrug9s
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=b58f6511a39fa02b71cc283f24ae796aaa6eece369cad307804ec3f40e2cfdb7
  Stored in directory: /tmp/pip-ephem-wheel-cache-xgavbj9k/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12


In [7]:
import os
import torch
import faiss
import json
import numpy as np
import clip
from PIL import Image

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

Running on: cpu


In [9]:
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("✅ CLIP model loaded.")

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 84.6MiB/s]


✅ CLIP model loaded.


In [10]:
index_path = f"{SAVE_DIR}/audio.index"
meta_path = f"{SAVE_DIR}/audio_meta.json"

In [11]:
assert os.path.exists(index_path), f"❌ Index file not found: {index_path}"
assert os.path.exists(meta_path), f"❌ Metadata file not found: {meta_path}"

index = faiss.read_index(index_path)
audio_meta = json.load(open(meta_path))
print(f"✅ Loaded FAISS index with {len(audio_meta)} audio embeddings.")

✅ Loaded FAISS index with 1 audio embeddings.


In [15]:
def encode_image_with_clip(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(image)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize
    return image_features.cpu().numpy().astype("float32")

# ------------------------------------------------
# 4️⃣ Query with an image
# ------------------------------------------------
image_path = "/content/test_scene/ai-generated-8630058.jpg"
image_emb = encode_image_with_clip(image_path)

In [21]:
D, I = index.search(image_emb, k=min(5, index.ntotal))

In [23]:
print("Vectors in FAISS index:", index.ntotal)
print("Entries in metadata:", len(audio_meta))
print("Indices returned:", I)

Vectors in FAISS index: 2
Entries in metadata: 1
Indices returned: [[1 0]]


In [22]:
!rm -f /content/drive/MyDrive/Project/audio.index
!rm -f /content/drive/MyDrive/Project/audio_meta.json

In [24]:
print("\n🔊 Top Retrieved Audio Matches:\n")

for rank, (score, idx) in enumerate(zip(D[0], I[0])):
    idx = int(idx)  # convert from numpy.int32/64 to plain int
    if 0 <= idx < len(audio_meta):
        filename = audio_meta[idx].get("filename", "unknown")
        print(f"Rank {rank+1}: {filename}  (similarity={score:.3f})")
    else:
        print(f"⚠️ Rank {rank+1}: invalid FAISS index {idx} (metadata only has {len(audio_meta)} entries)")


🔊 Top Retrieved Audio Matches:

⚠️ Rank 1: invalid FAISS index 1 (metadata only has 1 entries)
Rank 2: file_example_WAV_1MG.wav  (similarity=-0.064)
